In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# Read data in
#file = np.load('output.npz')
#xIn = file['x']
#yIn = file['y']
#print(xIn.shape)
imageShape = (1530,2720,3)
print(imageShape)

(1530, 2720, 3)


In [3]:
# setup the place holder input for the images
imageWidth = imageShape[1]
imageHeight = imageShape[0]

x = tf.placeholder(tf.float32, shape=[None, imageHeight, imageWidth, imageShape[2]])

# setup placeholder input for labels
y = tf.placeholder(tf.float32, shape=[None, 2])

# placeholder for batch norm training phase.
trainPhase = tf.placeholder(tf.bool)

print(x)
print(y)

Tensor("Placeholder:0", shape=(?, 1530, 2720, 3), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 2), dtype=float32)


In [4]:
# define the batch norm function for use.
decayRate = 0.99
betaInit = tf.zeros_initializer(dtype=tf.float32)
gammaInit = tf.ones_initializer(dtype=tf.float32)

# batchNormLayer
# Adds a batch normalization layer to to the filter.
# x - input tensor
# filterShape - shape of filter
# num - the number to not have the same variable name for the gamma and beta variables.
# filtType - the type of filter (conv, mult)
def batchNormLayer(x, numChannels, num, filtType='conv'):
    # assumed to be convlution filter

    #define weight variables
    gamma = tf.get_variable('gamma' + str(num), [numChannels], initializer=gammaInit)
    beta = tf.get_variable('beta' + str(num), [numChannels], initializer=betaInit)

    axes = []
    if filtType == 'mult':
        axes = [0]
    else:
        axes = [0,1,2]
    
    batch_mean, batch_variance = tf.nn.moments(x, axes)
    
    ema = tf.train.ExponentialMovingAverage(decay=decayRate)
    
    def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_variance])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_variance)

    mean, variance = tf.cond(trainPhase,
                        mean_var_with_update,
                        lambda: (ema.average(batch_mean), ema.average(batch_variance)))
    
    

    normed = tf.nn.batch_normalization(x, mean, variance, beta, gamma, 0.000001)
    return normed, gamma, beta, ema.average(batch_mean), ema.average(batch_variance)

In [5]:
numConvLayers = 0
# Define variable initilization
normInit = tf.truncated_normal_initializer(0,.05, dtype=tf.float32)
zeroInit = tf.constant_initializer(0.05, dtype=tf.float32)

# convLayer
# define convolutional layer with batch normalization, and max pooling
# @param x - the input tensor
# @param filterShape - the shape of the filter (height, width, num channels output)
# @param poolShape - the shape of the pooling (height width)
#
# @return layer, list of all variables
def convLayer(x, filterShape, poolShape):
    global numConvLayers
    inputChannels = x.shape[3]
    convFilt = tf.get_variable('filt' + str(numConvLayers), \
        [filterShape[0], filterShape[1], inputChannels, filterShape[2]], \
        initializer=normInit)
    bias = tf.get_variable('bias' + str(numConvLayers), \
        [filterShape[2]], initializer=zeroInit)
    
    # setup layer conv, batch norm, and pooling layers.
    logit = tf.nn.conv2d(x, convFilt, strides=[1,1,1,1], padding='SAME') + bias
    normed, gamma, beta, mean, variance = \
        batchNormLayer(logit, filterShape[2], numConvLayers)
    layer = tf.nn.relu(normed)
    pooled = tf.nn.max_pool(layer, \
                ksize=[1,poolShape[0], poolShape[1], 1], \
                strides=[1,poolShape[0], poolShape[1], 1], \
                padding='SAME')
    
    
    numConvLayers += 1
    return pooled, [convFilt, bias, gamma, beta, mean, variance]
    

# fullConnLayer
# Define a fully connected layer with batch normalization
# @param x - the input tensor
# @param numOutputNodes - number of output nodes
#
# @return outputLayer, list of all variables
def fullConnLayer(x, numOutputNodes):
    global numConvLayers
    inputChannels = x.shape[1]
    
    matFilt = tf.get_variable('filt' + str(numConvLayers), \
        [inputChannels, numOutputNodes], initializer=normInit)
    bias = tf.get_variable('bias' + str(numConvLayers), \
        [numOutputNodes], initializer=zeroInit)
    
    logit = tf.matmul(x, matFilt) + bias
    normed, gamma, beta, mean, variance = \
        batchNormLayer(logit, numOutputNodes, numConvLayers, 'mult')
    layer = tf.nn.relu(normed)
        
    numConvLayers += 1
    return layer, [matFilt, bias, gamma, beta, mean, variance]
    

In [6]:
variables = []

# define inference
layer1, tmp = convLayer(x, [3,3,128], [2,2])
print(layer1)
variables += tmp

layer2, tmp = convLayer(layer1, [5,5,64], [5,5])
variables += tmp
print(layer2)

layer3, tmp = convLayer(layer2, [3,3,128], [3,3])
variables += tmp
print(layer3)

layer4, tmp = convLayer(layer3, [7,7,64], [3,3])
variables += tmp
print(layer4)

layer5, tmp = convLayer(layer4, [5,5,64], [2,2])
variables += tmp
print(layer5)

layer6, tmp = convLayer(layer5, [7,7,32], [2,2])
variables += tmp
print(layer6)

sh = layer6.shape
flattened = tf.reshape(layer6, shape=[-1, sh[1]*sh[2]*sh[3]])
print(flattened)

full1, tmp = fullConnLayer(flattened, 1024)
variables += tmp
print(full1)

full2, tmp = fullConnLayer(flattened, 1024)
variables += tmp
print(full2)

outputMat = tf.get_variable('outputMat', [full2.shape[1], 2], initializer=normInit)
outputBias = tf.get_variable('outputBias', [2], initializer=zeroInit)

outputLogit = tf.matmul(full2, outputMat) + outputBias
print(outputLogit)
variables = variables + [outputMat, outputBias]

Tensor("MaxPool:0", shape=(?, 765, 1360, 128), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 153, 272, 64), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 51, 91, 128), dtype=float32)
Tensor("MaxPool_3:0", shape=(?, 17, 31, 64), dtype=float32)
Tensor("MaxPool_4:0", shape=(?, 9, 16, 64), dtype=float32)
Tensor("MaxPool_5:0", shape=(?, 5, 8, 32), dtype=float32)
Tensor("Reshape:0", shape=(?, 1280), dtype=float32)
Tensor("Relu_6:0", shape=(?, 1024), dtype=float32)
Tensor("Relu_7:0", shape=(?, 1024), dtype=float32)
Tensor("add_8:0", shape=(?, 2), dtype=float32)


In [7]:
# init tensorflow saver
saver = tf.train.Saver(variables)

In [8]:
# define loss function
crossEntropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=outputLogit)
loss = tf.reduce_mean(crossEntropy)

In [9]:
######## define training step

learningRate = 0.001

optimizer = tf.train.AdamOptimizer(learning_rate=learningRate)
trainStep = optimizer.minimize(loss)

###### define accuracy functions

# output of equals is integers
actualClass = tf.argmax(y, axis=1)
predictedClass = tf.argmax(outputLogit, axis=1)

equals = tf.equal(actualClass, predictedClass)

# cast integers to float for reduce mean to work correctly.
accuracy = tf.reduce_mean(tf.cast(equals, tf.float32))

In [10]:
# define batch code
# next_batch function will return a random set of images loaded from the given feed directory
import random
import glob
import matplotlib.pyplot as im
from skimage.transform import resize
import os
import labelReader

# read in all image names
feed_dir = '../labelingTool/feed/'
imageFilenames = glob.glob(feed_dir + '*.jpg')
# read in labels csv file
labelsFromFilenames = labelReader.readLabelsDict('../labelingTool/labels.csv')

############################################## Variables to set batch sizes
numValidationImages = 50
batchSize = 30
#############################################

# pull out small subset of images to use for validation
validationFilenames = imageFilenames[0:numValidationImages]
trainFilenames = imageFilenames[numValidationImages:]



# this function goes through and pulls random
# parts of the training data out
# @param filenames - a list of all possible filenames to pull from
# @param batchSize - the list of batchSize 
#
# @return - images, labels as numpy arrays
def pullRandomBatch(filenames, batchSize):
    # select random indcies of filenames
    indicies = np.random.choice(len(filenames), batchSize, replace=False)
    
    # Read in all filenames of 
    batchFilenames = [filenames[i] for i in indicies]
    
    return createBatch(batchFilenames)
    

def createBatch(batchFilenames):
    # create vector of images and labels
    images = np.empty((batchSize, imageShape[0], imageShape[1], imageShape[2]))
    labels = np.zeros((batchSize, 2))
    
    # Go through each image filename and read in the image, and put
    # it into a label.
    # If the the filesize is not correct, than resize the image.
    for i in range(len(batchFilenames)):
        image = im.imread(batchFilenames[i])
        
        # resize image if wrong size.
        if image.shape != imageShape:
            image = resize(image, imageShape)
        
        # set image into numpy array
        images[i] = image
        filenameBase = os.path.basename(batchFilenames[i])
        
        # set label for the image.
        label = labelsFromFilenames[filenameBase]
        if label == 'noPeople':
            labels[i][0] = 1.0
        elif label == 'people':
            labels[i][1] = 1.0
        else:
            print('Error, the image: '+ filename + 'is not in the labels')
    
    return images, labels

train, label = pullRandomBatch(trainFilenames, batchSize)
print(train.shape)
print(label.shape)

validData, labelLabels = createBatch(validationFilenames)
print(validData.shape)

/Users/Ian/anaconda/envs/flow/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/Ian/anaconda/envs/flow/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


(30, 1530, 2720, 3)
(30, 2)
(30, 1530, 2720, 3)


In [ ]:
sess = tf.Session()

# comment the restore call and uncoment the global initializer to restart the training the process.
#saver.restore(sess, 'V6')
sess.run(tf.global_variables_initializer())

In [ ]:
# define training loop
numIterations = 2000
numToValidate = 50
numToSave = 1000000



lossSum = 0.0
for i in range(numIterations):
    train, label = pullRandomBatch(trainFilenames, batchSize)
    feed = {x: train, y: label, trainPhase: True}
    
    lossSum += sess.run(loss, feed_dict=feed)
    sess.run(trainStep, feed_dict=feed)
    
    if (i+1) % numToSave == 0: 
        # save the current model
        #saver.save(sess, 'layer2/layer2ShortFilters', global_step=i)
        xyz =1
    
    if i % numToValidate == 0: 
        feed = {x: validData, y: labelLabels, trainPhase: False}
        
        acc = sess.run(accuracy, feed_dict=feed)
        print('iteration num: ' + str(i))
        print('Validation accuracy = ' + str(acc))
        print('Avg loss = ' + str(lossSum / numToValidate))
        lossSum = 0.0



/Users/Ian/anaconda/envs/flow/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/Ian/anaconda/envs/flow/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
